### get list invoice numbers actually downloaded

In [4]:
from pathlib import Path
import pandas as pd
import openpyxl

path_invoice = r'C:/Users/john.tan/Downloads/'

def find_files(dir, ext):
    list_files = []
    for path in Path(dir).glob('**/*'+ext):  # iterate over all
        if path.suffix.lower() == ext:  # check if the path pattern matches
            list_files.append(path)            
    return list_files
        
list_path_files = find_files(path_invoice+"esker_merged/", ".pdf")

def extract_invoice_numbers(list_path_files):
    list_only_invoice_numbers = []
    for path in list_path_files:
        # Extract the file name from the path
        path_str = str(path)
        file_name = path_str.split('\\')[-1]
                
        invoice_number = file_name.replace('_merged.pdf', '') # Remove '_merged.pdf'
        list_only_invoice_numbers.append(invoice_number)
    return list_only_invoice_numbers

list_invoice_numbers_0 = extract_invoice_numbers(list_path_files)
#convert elem to str
list_invoice_numbers = ["".join(map(str, elem)) for elem in list_invoice_numbers_0]


df_invoice_original = pd.read_excel(path_invoice+'ALL_VENDORS.xlsx', sheet_name='all_vendors', engine='openpyxl')
list_invoice_original_0 = df_invoice_original['invoice'].tolist()
list_invoice_original = [str(elem) for elem in list_invoice_original_0]
list_invoice_original = [elem.replace('/', '_' ) for elem in list_invoice_original]

list_invoice_common = list(set(list_invoice_original) & set(list_invoice_numbers))
list_invoice_diff = list(set(list_invoice_original) - set(list_invoice_numbers))

def convert_invoice_numbers(invoice_list):
    converted_list = []
    for item in invoice_list:
        try:            
            converted_item = int(item) # Try to convert the item to an integer
        except ValueError:            
            converted_item = item # If conversion fails, keep it as a string
        converted_list.append(converted_item)
    return converted_list

list_converted_diff = convert_invoice_numbers(list_invoice_diff)
df_invoice_actually_downloaded = df_invoice_original[~df_invoice_original['invoice'].isin(list_converted_diff)]
df_invoice_rename = df_invoice_actually_downloaded[['invoice_name', 'invoice', 'invoice_date', 'company_code']]

with pd.ExcelWriter(path_invoice+'esker_merged/invoice_rename.xlsx') as writer_rename:
    df_invoice_rename.to_excel(writer_rename, sheet_name='invoice_rename', index=False)

len(df_invoice_rename)



90